# Student Loan Risk with Deep Learning

In [26]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential, load_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [2]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [3]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [4]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [6]:
# Define the target set y using the credit_ranking column
y = loans_df['credit_ranking']

# Display a sample of y
y

0       0
1       0
2       0
3       1
4       0
       ..
1594    0
1595    1
1596    1
1597    0
1598    1
Name: credit_ranking, Length: 1599, dtype: int64

In [8]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.drop(columns='credit_ranking')

# Review the features DataFrame
X

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4
...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2


### Step 3: Split the features and target sets into training and testing datasets.


In [9]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [14]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test.columns)

---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [17]:
# Define the the number of inputs (features) to the model
num_inputs = len(X_train_scaled_df.columns)

# Review the number of features
num_inputs

11

In [10]:
# Define the number of hidden nodes for the first hidden layer


# Define the number of hidden nodes for the second hidden layer


# Define the number of neurons in the output layer


In [19]:
# Create the Sequential model instance
model = Sequential()

# Add the first hidden layer
model.add(Dense(units=64, activation="relu", input_dim=num_inputs))

# Add the second hidden layer
model.add(Dense(units=32, activation="relu"))


# Add the output layer to the model specifying the number of output neurons and activation function
model.add(Dense(units=1, activation="sigmoid"))

C:\Users\jnpre\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [20]:
# Display the Sequential model summary
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 64)                  │             768 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,881 (11.25 KB)

 Trainable params: 2,881 (11.25 KB)

 Non-trainable params: 0 (0.00 B)

### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [22]:
# Compile the Sequential model
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [23]:
# Fit the model using 50 epochs and the training data
fit_model = model.fit(X_train_scaled_df, y_train, epochs=50)

Epoch 1/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 798us/step - accuracy: 0.6173 - loss: 0.6545 
Epoch 2/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 703us/step - accuracy: 0.7243 - loss: 0.5620
Epoch 3/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 703us/step - accuracy: 0.7348 - loss: 0.5320
Epoch 4/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 649us/step - accuracy: 0.7580 - loss: 0.5138
Epoch 5/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 622us/step - accuracy: 0.7606 - loss: 0.5012
Epoch 6/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 649us/step - accuracy: 0.7614 - loss: 0.4868
Epoch 7/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 649us/step - accuracy: 0.7552 - loss: 0.4837
Epoch 8/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 595us/step - accuracy: 0.7578 - loss: 0.4866
Epoch 9/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 622us/step - accuracy: 0.7840 - loss: 0.4620
Epoch 10/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 649us/step - accuracy: 0.7730 - loss: 0.4580
Epoch 11/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 649us/step - accuracy: 0.7998 - loss: 0.4412
Epoch 12/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 649us/ste

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [24]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = model.evaluate(X_test_scaled_df, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


13/13 - 0s - 7ms/step - accuracy: 0.7375 - loss: 0.5537
Loss: 0.5537080764770508, Accuracy: 0.737500011920929


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [25]:
# Set the model's file path


# Export your model to a keras file
model.save('student_loans.keras')

---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [27]:
# Set the model's file path


# Load the model to a new object
loaded_model = load_model('./student_loans.keras')

### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [28]:
# Make predictions with the test data
predicted = loaded_model.predict(X_test_scaled_df)

# Display a sample of the predictions
predicted

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


array([[0.08409262],
       [0.3239475 ],
       [0.75046325],
       [0.7103082 ],
       [0.9638173 ],
       [0.9377589 ],
       [0.9682681 ],
       [0.01303144],
       [0.53348136],
       [0.21428551],
       [0.9794178 ],
       [0.13322215],
       [0.6514565 ],
       [0.93584305],
       [0.78271747],
       [0.2718463 ],
       [0.93507344],
       [0.20032582],
       [0.79151154],
       [0.43629223],
       [0.58871615],
       [0.8586249 ],
       [0.22985926],
       [0.9829354 ],
       [0.16049369],
       [0.9653471 ],
       [0.90772295],
       [0.5227322 ],
       [0.10098507],
       [0.9138907 ],
       [0.6620076 ],
       [0.9831415 ],
       [0.2328153 ],
       [0.9874151 ],
       [0.18041025],
       [0.52720654],
       [0.20266639],
       [0.6267998 ],
       [0.9907432 ],
       [0.04206148],
       [0.91394234],
       [0.03791886],
       [0.04159462],
       [0.9798834 ],
       [0.04936935],
       [0.7668985 ],
       [0.14963575],
       [0.521

In [34]:
# Save the predictions to a DataFrame and round the predictions to binary results
pred_df = pd.DataFrame(predicted).round().astype(int)
pred_df

,0
0,0
1,0
2,1
3,1
4,1
...,...
395,1
396,0
397,1
398,0


### Step 4: Display a classification report with the y test data and predictions

In [36]:
# Print the classification report with the y test data and predictions
print(classification_report(y_test, pred_df))

              precision    recall  f1-score   support

           0       0.70      0.77      0.73       188
           1       0.77      0.71      0.74       212

    accuracy                           0.74       400
   macro avg       0.74      0.74      0.74       400
weighted avg       0.74      0.74      0.74       400



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**


It's necessary for the data collected to match the data used for training. All of the features used for the model must be collected, and features that cna't be collected should be estimated

**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**

Collaborative filtering, because it compares the student to other students

**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**

1. Interest rates are subject to change based on time, so timeseries data may be necessary, and collecting data related to interest rates
2. The economic conditions after students enter the workforce affects ability to repay, and must be predicted at the time the loan is issued